In [1]:
import pandas as pd
import numpy as np
import math


columns = ['X', 'Y']

df = {'X':[2, 2, 8, 5, 7, 6, 1, 4],
      'Y':[10, 5, 4, 8, 5, 4, 2, 9]}

df = pd.DataFrame(df, columns=columns)


centroid_labels = ['c1', 'c2', 'c3']

centroids = {'X': [8, 5, 1],
             'Y': [4, 8, 2]}

centroids = pd.DataFrame(centroids, columns=columns).astype(float)


print(df)

   X   Y
0  2  10
1  2   5
2  8   4
3  5   8
4  7   5
5  6   4
6  1   2
7  4   9


1)  Trace the k-means algorithm to find three clusters in the given dataset.  Asume that the Euclidean distance is used and the initially randomly selected centroids are (8, 5), (5, 8), and (1, 2) respectively.

1a)  Find the cluster for each data point with current centroids

In [2]:
def get_centroid_distance(df, centroids):
    return_df = pd.DataFrame(np.zeros((len(df.index), len(centroid_labels))), columns=centroid_labels)
    
    for i in range(len(df.index)):
        temp_distance = []
        
        for j in range(len(centroid_labels)):
            dist = 0
            
            for col in columns:
                dist += math.pow((df.iloc[i][col] - centroids.iloc[j][col]), 2)
                
            dist = math.sqrt(dist)
            
            return_df.iloc[i][centroid_labels[j]] = dist
            
    return return_df
            
def k_means(df, centroids):
    round = 1
    prev_cents = centroids.copy()
    prev_cents['X'] = 0
    
    print("\nK-Means Clustering\n")
    
    print("Points:")
    print(df)
    
    print("\nCentroids")
    print(centroids)
    
    while not centroids.equals(prev_cents):
        prev_cents = centroids.copy()

        print("\nRound {0}:".format(round))
        round += 1
        
        cluster = []

        distances = get_centroid_distance(df, centroids)

        df['cluster'] = distances.idxmin(axis=1)

        print("\nPoint Distances to Centroids")
        print(distances)
        
        print("\nPoints with labeled Centroids")
        print(df)

        for cent in centroid_labels:
            x = 0
            y = 0
            
            cluster_points = df.loc[df['cluster'] == cent]
            
            x = cluster_points['X'].mean()
            y = cluster_points['Y'].mean()
            
            centroids.iloc[centroid_labels.index(cent)]['X'] = x
            centroids.iloc[centroid_labels.index(cent)]['Y'] = y
        
        print("\nNew Centroids")
        print(centroids)
        

def print_clusters(df):
    print("\nFinal Clusters:\n")
    
    for cent in centroid_labels:
        cluster_points = df.loc[df['cluster'] == cent]
        
        print("Cluster {0}:".format(cent))
        print(cluster_points)
        print()
    
    
k_means(df, centroids)

print_clusters(df)


K-Means Clustering

Points:
   X   Y
0  2  10
1  2   5
2  8   4
3  5   8
4  7   5
5  6   4
6  1   2
7  4   9

Centroids
     X    Y
0  8.0  4.0
1  5.0  8.0
2  1.0  2.0

Round 1:

Point Distances to Centroids
         c1        c2        c3
0  8.485281  3.605551  8.062258
1  6.082763  4.242641  3.162278
2  0.000000  5.000000  7.280110
3  5.000000  0.000000  7.211103
4  1.414214  3.605551  6.708204
5  2.000000  4.123106  5.385165
6  7.280110  7.211103  0.000000
7  6.403124  1.414214  7.615773

Points with labeled Centroids
   X   Y cluster
0  2  10      c2
1  2   5      c3
2  8   4      c1
3  5   8      c2
4  7   5      c1
5  6   4      c1
6  1   2      c3
7  4   9      c2

New Centroids
          X         Y
0  7.000000  4.333333
1  3.666667  9.000000
2  1.500000  3.500000

Round 2:

Point Distances to Centroids
         c1        c2        c3
0  7.557189  1.943651  6.519202
1  5.044249  4.333333  1.581139
2  1.054093  6.616478  6.519202
3  4.176655  1.666667  5.700877
4  0.666667  5.2

2)  Trace the PAM algorithm to find three clusters in the given dataset.  Assume that the Euclidean distance is used and the randomly selected initial medoids are (2, 5), (5, 8), and (1, 2) respectively.

I did not do question #2.

3)  Write a python notebook to use scikit-learn cluster.KMeans to cluster the data in hwk08.csv.  Try to cluster the data using different subset of the columns, and different number of clusters.  For an additional 5 points bonus, plot the clusters.

In [3]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from itertools import combinations 

df = pd.read_csv('hwk08.csv')

col_combos = []

results = pd.DataFrame(None, columns=['k', 'cols', 'SSE'])

for i in range(2, 5):
    col_combos = col_combos + list(combinations(['A', 'B', 'C', 'D'], i))

for cols in col_combos:
    X = df[list(cols)]
    
    for i in range(1, 11):
        km = KMeans(n_clusters=i, init='random', n_init=10, max_iter=300, tol=1e-04, random_state=0)

        y_km = km.fit_predict(X)
        
        # (num_clusters, cols_list, SSE)
        results = results.append({'k':i, 'cols':cols, 'SSE':km.inertia_}, ignore_index = True)
        
lowest = results[results['SSE']==results['SSE'].min()]    

In [4]:
print("Columns C and D seem to be the most useful, consistently producing the lowest SSE score.")
print("A larger K results in smaller clusters and a lower SSE.")
print("Using k=50 produces the following best result:")

list1 = lowest.values.tolist()[0]

print("k={0}".format(list1[0]))
print("columns = {0}".format(list(list1[1])))
print("SSE = {0}".format(list1[2]))

Columns C and D seem to be the most useful, consistently producing the lowest SSE score.
A larger K results in smaller clusters and a lower SSE.
Using k=50 produces the following best result:
k=10
columns = ['C', 'D']
SSE = 283.1376252579788


There doesn't seem to be a great way to automate selection of the columns and k-value.  I think better analysis would look at graphs of the clusters and the elbow curves.